In [1]:
#%reload_ext Cython

In [2]:
#import typing
#import copy
#import pdb
#from IPython.display import clear_output
#import numpy as np
#import time
#from colorama import Fore
#import sys
#import cProfile
#from multiprocessing import Pool
#import chess 
#import chess.uci
#import tensorflow as tf
#import random
#import boardclass
import test
import boardclass

In [3]:
path = "../extracted/good_games.txt"
tested_upto = 118
#boardclass.file_to_move_lists(path,move_size=5)
troubling_moves,troubling_index = test.compare_boards_with_file(path,index=tested_upto)
print(troubling_moves)
print(troubling_index)

118 of of 29168
119 of of 29168
120 of of 29168
121 of of 29168
122 of of 29168
123 of of 29168
124 of of 29168
125 of of 29168
126 of of 29168
127 of of 29168


KeyboardInterrupt: 

In [ ]:
test.start_game(-1,troubling_moves,watching=True,fast_play=True,fast_play_count=troubling_index,clear=False,set_pdb=True)